In [1]:
import sys
import itertools
import math
import numpy as np
import json
import warnings
from datetime import datetime
import matplotlib.pyplot as plt

import experimental.utils as utils
from experimental.agent import PIDTuningAgent
from experimental.environment import PIDTuningEnvironment
from experimental.runner import Runner
from experimental.runner_opt import Runner_opt

warnings.filterwarnings("ignore")

In [2]:
#Open json file
f = open(f'config/testcase_synt_1.json')
param_dict = json.load(f)

horizon = param_dict['horizon']
n_trials = param_dict['n_trials']
sigma = param_dict['noise_sigma']

n = param_dict['n']
p = param_dict['p']
m = param_dict['m']

A = np.array(param_dict['A'])
b = np.array(param_dict['B'])
c = np.array(param_dict['C'])

#Step signal
y_0 = 1

In [3]:
#Define dictionary for the errors of the algorithms
optimal = "optimal"
pidtuning = "pidtuning"
ziegler_nichols = "ziegler_nichols"
alg_list = [optimal, pidtuning, ziegler_nichols]
errors = {alg: np.zeros((n_trials, horizon)) for alg in alg_list}

#Define noises
np.random.seed(1)
noise = np.random.normal(0, sigma, (n_trials, horizon, n))
out_noise = np.random.normal(0, sigma, (n_trials, horizon, m))

In [4]:
#Define range of possible PID parameters
log_space = np.logspace(0, 1, num=15, base=10)

K_P_range_start = 0.0
K_P_range_end = 1.5
K_P_range = (log_space - log_space.min()) / (log_space.max() - log_space.min()) *\
      (K_P_range_end - K_P_range_start) + K_P_range_start

K_I_range_start = 0.1
K_I_range_end = 2.0
K_I_range = (log_space - log_space.min()) / (log_space.max() - log_space.min()) *\
      (K_I_range_end - K_I_range_start) + K_I_range_start

K_D_range_start = 0.0
K_D_range_end = 0.8
K_D_range = (log_space - log_space.min()) / (log_space.max() - log_space.min()) *\
      (K_D_range_end - K_D_range_start) + K_D_range_start

In [5]:
#Build list of ammissible PID parameters
pid_actions = []
for K in list(itertools.product(K_P_range, K_I_range, K_D_range)):
    bar_A = utils.compute_bar_a(A, b, c, K)
    if (np.max(np.absolute(np.linalg.eigvals(bar_A))) < 0.4): 
        pid_actions.append(np.array(K).reshape(3,1))

pid_actions = np.array(pid_actions)
n_arms = pid_actions.shape[0]
print(n_arms)

8


### Optimal algorithm

In [6]:
#Run optimal algorithm

env = PIDTuningEnvironment(A, b, c, n, p, m, y_0, horizon, noise, out_noise, n_trials)

all_errors = np.zeros((n_arms, n_trials, horizon))
all_SSE = np.zeros((n_arms, n_trials))
K_opt_idx = np.zeros(n_trials)
K_opt = np.zeros((n_trials, 3, 1))
i = 0
for i, K in enumerate(pid_actions):
    print("Running simulation ", i)
    runner_opt = Runner_opt(env, n_trials, horizon, 3, n_arms, pid_actions)
    all_errors[i] = runner_opt.perform_simulations(K)
    for trial_i in range(n_trials):
        all_SSE[i] = np.sum(np.power(all_errors[i, trial_i],2))
    i += 1
for trial_i in range(n_trials):
    K_opt_idx[trial_i] = np.argmin(all_SSE[:, trial_i])
    K_opt[trial_i] = pid_actions[int(K_opt_idx[trial_i])]
    errors[optimal][trial_i,:] = all_errors[int(K_opt_idx[trial_i]), trial_i, :]

print(K_opt)

Running simulation  0
Running simulation  1
Running simulation  2
Running simulation  3
Running simulation  4
Running simulation  5
Running simulation  6
Running simulation  7
[[[1.24723816]
  [0.98233354]
  [0.03462182]]

 [[1.24723816]
  [0.98233354]
  [0.03462182]]

 [[1.24723816]
  [0.98233354]
  [0.03462182]]]


In [ ]:
#Plot K_P and SSE, fixed K_I and K_D
idx = []
for i, K in enumerate(pid_actions):
    if (K[1] == K_opt[0][1] and K[2] == K_opt[0][2]):
        idx.append(i)
y_plot = []
x_plot = []
for i in idx:
    y_plot.append(all_SSE[i][0])
    x_plot.append(pid_actions[i][0][0])

print(K_opt[0][0][0])

plt.figure()
plt.plot(np.array(x_plot), np.array(y_plot))
plt.xlabel('K_P')
plt.ylabel('SSE')
plt.grid(True)
plt.show()

In [ ]:
#Plot K_D and SSE, fixed K_P and K_I
idx = []
for i, K in enumerate(pid_actions):
    if (K[0] == K_opt[0][0] and K[1] == K_opt[0][1]):
        idx.append(i)
y_plot = []
x_plot = []
for i in idx:
    y_plot.append(all_SSE[i][0])
    x_plot.append(pid_actions[i][2][0])

print(K_opt[0][2][0])

plt.figure()
plt.plot(np.array(x_plot), np.array(y_plot))
plt.xlabel('K_D')
plt.ylabel('SSE')
plt.grid(True)
plt.show()


In [ ]:
#Plot K_I parameters and SSE, fixed K_P and K_D
idx = []
for i, K in enumerate(pid_actions):
    if (K[0][0] == K_opt[0][0][0] and K[2][0] == K_opt[0][2][0]):
        idx.append(i)
y_plot = []
x_plot = []
for i in idx:
    y_plot.append(all_SSE[i][0])
    x_plot.append(pid_actions[i][1][0])

print(K_opt[0][1][0])

plt.figure()
plt.plot(np.array(x_plot), np.array(y_plot))
plt.xlabel('K_I')
plt.ylabel('SSE')
plt.grid(True)
plt.show()

### Ziegler-Nichols open loop

In [ ]:
#Load .mat file containing step response
import scipy.io

mat = scipy.io.loadmat('step_response.mat')
data = mat['data']

y_zn = data[:, 1]
print(np.shape(y_zn))

In [ ]:
#Compute regret for Ziegler-Nichols algorithm

inst_regret_zn = np.zeros(horizon)
cum_regret_zn = np.zeros(horizon)

errors[ziegler_nichols][0] = y_0 - y_zn
inst_regret_zn =  errors[ziegler_nichols][0]**2 - np.mean(errors[optimal])**2
cum_regret_zn = np.cumsum(inst_regret_zn)
cum_regret_std_zn = np.std(cum_regret_zn, axis=0)
print(np.shape(cum_regret_zn))

### PID Tuning algorithm

In [7]:
#Upper bound for relevant quantities
action_max = pid_actions[np.argmax([np.linalg.norm(np.array(K), 2) for K in pid_actions])]
K_val = np.linalg.norm(action_max, 2)
b_val = np.linalg.norm(b, 2)
c_val = np.linalg.norm(c, 2)
spectral_rad_ub = max(np.linalg.eigvals(A))
phi_a_ub = utils.spectr(A)
y_0 = 1


#Upper bound for noise
noise_norm = []
for trial_i in range(n_trials):
    for t in range(horizon):
        noise_norm.append(np.linalg.norm(noise[trial_i, t, :]))
        noise_norm.append(np.linalg.norm(out_noise[trial_i, t, :]))
noise_ub = max(np.array(noise_norm))


#Upper bound for spectral radius of matrix bar_A
spectral_rad_list = []
for K in pid_actions:
    bar_A = utils.compute_bar_a(A, b, c, K)
    spectral_rad_list.append(np.max(np.absolute(np.linalg.eigvals(bar_A))))

spectral_rad_bar_ub = np.max(np.array(spectral_rad_list))
bar_A = utils.compute_bar_a(A, b, c, pid_actions[np.argmax(np.array(spectral_rad_list))])
phi_bar_a_ub = utils.spectr(bar_A)

print(spectral_rad_bar_ub)

0.395065237075945


In [8]:
#Create file for PIDTuning algorithm checkpoints
#It saves the error at each time, for each simulation
#It works even with interruptions
temp = np.zeros((n_trials, horizon))
np.save("pid_tuning_errors.npy", temp)

In [9]:
#Running PIDTuning
agent = PIDTuningAgent(n_arms, pid_actions, horizon,
                            np.log(horizon), b_val, c_val, K_val, phi_a_ub, phi_bar_a_ub, y_0,
                            spectral_rad_ub, spectral_rad_bar_ub, noise_ub, sigma)
env = PIDTuningEnvironment(A, b, c, n, p, m, y_0, horizon, noise, out_noise, n_trials)
print('Running PID Tuning')
runner = Runner(env, agent, n_trials, horizon, 3, n_arms, pid_actions)
errors[pidtuning] = runner.perform_simulations()

Running PID Tuning


  0%|          | 0/3 [00:00<?, ?it/s]

Simulation  0
Time  0
pull first arm
Time  1
Time  2
Time  3
Time  4
update
Time  5
pull arm
estimate action
Time  6
Time  7
Time  8
update
Time  9
pull arm
estimate action
Time  10
Time  11
Time  12
update
Time  13
pull arm
estimate action
Time  14
Time  15
Time  16
update
Time  17
pull arm
estimate action
Time  18
Time  19
Time  20
update
Time  21
pull arm
estimate action
Time  22
Time  23
Time  24
update
Time  25
pull arm
estimate action
Time  26
Time  27
Time  28
update
Time  29
pull arm
estimate action
Time  30
Time  31
Time  32
update
Time  33
pull arm
estimate action
Time  34
Time  35
Time  36
update
Time  37
pull arm
estimate action
Time  38
Time  39
Time  40
update
Time  41
pull arm
estimate action
Time  42
Time  43
Time  44
update
Time  45
pull arm
estimate action
Time  46
Time  47
Time  48
update
Time  49
pull arm
estimate action
Time  50
Time  51
Time  52
update
Time  53
pull arm
estimate action
Time  54
Time  55
Time  56
update
Time  57
pull arm
estimate action
Time  58
Ti

In [ ]:
matrix = agent.V_t
eigenvalues = np.linalg.eig(matrix)
for eig in eigenvalues:
                if (eig.real < 0.1 and eig.imag < 0.1):
                    print(eig)

### Regret

In [ ]:
#Compute regret for PIDTuning
inst_regret = np.zeros((n_trials, horizon))
cum_regret = np.zeros((n_trials, horizon))

inst_regret =  errors[pidtuning]**2 - errors[optimal]**2
for trial_i in range(n_trials):
    cum_regret[trial_i] = np.cumsum(inst_regret[trial_i])
cum_regret_mean = np.mean(cum_regret, axis=0)
cum_regret_std = np.std(cum_regret, axis=0) / np.sqrt(n_trials)

In [ ]:
#Save results 
np.savez('arrays.npz',K_opt_idx=K_opt_idx, K_opt=K_opt, errors=errors,\
          inst_regret=inst_regret, cum_regret=cum_regret, cum_regret_mean=cum_regret_mean,\
          cum_regret_std=cum_regret_std)

In [ ]:
#Load results
data = np.load('arrays.npz', allow_pickle=True)
K_opt = data['K_opt']
inst_regret = data['inst_regret']
cum_regret = data['cum_regret']
cum_regret_mean = data['cum_regret_mean']
cum_regret_std = data['cum_regret_std']

In [11]:
data = np.load('pid_tuning_errors.npy', allow_pickle=True)
print(np.shape(data))
print(data[2])

(3, 100)
[ 1.00013349e+00 -4.24864502e-02  4.04249490e-01  2.83403873e-02
  1.66941007e-01  3.30403068e-02  7.58424556e-02  1.90035814e-02
  3.67576204e-02  7.56344575e-03  1.92079993e-02  4.28827349e-03
  8.41346979e-03  3.81548785e-03  3.20252763e-03  2.47748253e-03
  2.60970648e-03 -8.28079890e-04  7.57776248e-04 -1.37956864e-03
  2.94474626e-03 -2.40577227e-03  5.74949389e-03 -4.77450171e-03
  2.06819397e-03 -1.28571695e-03  4.83697648e-04 -1.81860393e-04
  4.18891594e-04 -2.01062123e-03  2.70226132e-03 -7.19012436e-04
 -2.18180002e-03  1.21082544e-03 -1.39628624e-04  1.07909046e-03
 -9.42347417e-04  1.00604288e-03  2.05114695e-03 -3.00730397e-03
  1.82041285e-03  7.92883497e-04 -1.73405221e-03  8.76186570e-04
 -8.10198087e-04 -3.86817850e-03  5.29508827e-03  3.57745966e-05
 -9.13557382e-04  1.50729261e-05  8.40746192e-05 -1.72944414e-03
  7.87287599e-04 -1.00179140e-03  1.91086947e-03 -7.37938619e-04
  2.57823685e-03 -2.68355355e-04 -2.80644055e-03  4.79335455e-04
  1.65434648e-03

In [ ]:
#Plotting
plt.figure(figsize=(10, 6))

plt.plot(cum_regret_mean, label='PIDTuning')
plt.fill_between(range(len(cum_regret_mean)), 
                 cum_regret_mean - cum_regret_std, 
                 cum_regret_mean + cum_regret_std, 
                 color='b', alpha=0.2, label='Standard Error PIDTuning')

plt.plot(cum_regret_zn, label='Ziegler-Nichols')
plt.fill_between(range(len(cum_regret_zn)), 
                 cum_regret_zn - cum_regret_std_zn, 
                 cum_regret_zn + cum_regret_std_zn, 
                 alpha=0.2, label='Standard Error ZN')

plt.xlabel('Time Horizon')
plt.ylabel('Cumulative Regret')
#plt.axis('equal')
plt.legend()
plt.grid(True)
plt.show()